In [37]:
import pandas as pd
import tushare as ts
from sqlalchemy import create_engine
import time
import datetime
import json
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,LSTM,Dropout
import math
from sklearn.metrics import mean_squared_error

In [38]:
with open('./parameters.json','r') as f:
    p=json.load(f)
    user = p['user']
    port = p['port']
    psw = p['password']
    host = p['host']
    token=p['TU_share_pro_taken']
    cnnstr = "mysql://" + user + ":" + psw + "@" + host + ":" + str(port) + "/stock?charset=utf8&use_unicode=1"
engine_ts = create_engine(cnnstr)
ts.set_token(token)

In [128]:
df = ts.pro_bar(ts_code='600678.SH',
                            asset='E', adj='qfq', freq='D')
df=df[['trade_date','open','high','low','close','vol']]


In [129]:
df.drop(df[np.isnan(df['close'])].index, inplace=True)  # 删除收盘价为空的所有行。
df.sort_values(by='trade_date',inplace=True)
df.head(5)

,trade_date,open,high,low,close,vol
4436,20000324,3.6733,3.6784,3.5198,3.5198,21221.0
4435,20000327,3.5301,3.6784,3.5301,3.6682,27297.0
4434,20000328,3.7091,3.7501,3.6836,3.7143,33903.0
4433,20000329,3.7245,3.7347,3.5915,3.6375,19684.0
4432,20000330,3.6324,3.6733,3.6119,3.6733,15926.0


In [105]:
moving_ndays_close=[]
for i in range(0, len(df) - 5):
    ndays = df.iloc[i:i + 5, 3]
    moving_ndays_close.append(np.mean(ndays))
# df.drop(df.tail(5).index, inplace=True)
df['moving_ndays_close'] = moving_ndays_close+list(np.zeros(5))
df.tail(10)

,trade_date,open,high,low,close,vol,moving_ndays_close
9,20201021,6.59,6.59,6.45,6.47,41759.28,6.228
8,20201022,6.46,6.46,6.25,6.35,51765.98,6.170
7,20201023,6.34,6.41,6.22,6.24,38293.43,6.130
6,20201026,6.25,6.25,6.11,6.16,38059.02,6.068
5,20201027,6.14,6.32,6.11,6.28,42970.74,6.000
4,20201028,6.28,6.34,6.16,6.23,31757.01,0.000
3,20201029,6.12,6.25,6.05,6.19,47738.27,0.000
2,20201030,6.20,6.22,5.91,5.94,70970.08,0.000
1,20201102,5.92,5.96,5.77,5.81,65548.92,0.000
0,20201103,5.80,6.14,5.80,6.09,80188.31,0.000


In [106]:
# 加工训练数据
df1= df[df['trade_date']<='20201023']
df1.tail(5)

,trade_date,open,high,low,close,vol,moving_ndays_close
11,20201019,6.64,6.75,6.63,6.66,48175.74,6.412
10,20201020,6.64,6.65,6.51,6.61,40515.06,6.308
9,20201021,6.59,6.59,6.45,6.47,41759.28,6.228
8,20201022,6.46,6.46,6.25,6.35,51765.98,6.170
7,20201023,6.34,6.41,6.22,6.24,38293.43,6.130


In [108]:
df1.drop('trade_date', axis=1, inplace=True)
ar = np.array(df1.tail(10))
ar

C:\Users\ny7146\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\core\frame.py:3694: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


array([[6.680000e+00, 6.870000e+00, 6.670000e+00, 6.840000e+00,
        6.293249e+04, 6.640000e+00],
       [6.860000e+00, 6.900000e+00, 6.750000e+00, 6.790000e+00,
        3.753800e+04, 6.632000e+00],
       [6.800000e+00, 6.800000e+00, 6.620000e+00, 6.650000e+00,
        4.890048e+04, 6.584000e+00],
       [6.720000e+00, 6.730000e+00, 6.600000e+00, 6.610000e+00,
        3.077361e+04, 6.550000e+00],
       [6.610000e+00, 6.640000e+00, 6.560000e+00, 6.620000e+00,
        3.223879e+04, 6.480000e+00],
       [6.640000e+00, 6.750000e+00, 6.630000e+00, 6.660000e+00,
        4.817574e+04, 6.412000e+00],
       [6.640000e+00, 6.650000e+00, 6.510000e+00, 6.610000e+00,
        4.051506e+04, 6.308000e+00],
       [6.590000e+00, 6.590000e+00, 6.450000e+00, 6.470000e+00,
        4.175928e+04, 6.228000e+00],
       [6.460000e+00, 6.460000e+00, 6.250000e+00, 6.350000e+00,
        5.176598e+04, 6.170000e+00],
       [6.340000e+00, 6.410000e+00, 6.220000e+00, 6.240000e+00,
        3.829343e+04, 6.130

In [123]:
df2=df
df.loc[:'20000328',:]
# df2.set_index(df2['trade_date'],inplace=True,drop=True)
# df2.drop('trade_date',axis=1,inplace=True)

,open,high,low,close,vol,moving_ndays_close
trade_date,,,,,,
20000324,3.6733,3.6784,3.5198,3.5198,21221.0,3.58738
20000327,3.5301,3.6784,3.5301,3.6682,27297.0,3.59456
20000328,3.7091,3.7501,3.6836,3.7143,33903.0,3.60478


In [124]:
scaler = MinMaxScaler(feature_range=(0,1))
data = scaler.fit_transform(df)


In [127]:
df=pd.DataFrame(data)
df

,0,1,2,3,4,5
0,0.084429,0.081171,0.085268,0.081180,0.014679,0.122070
1,0.079927,0.081171,0.085606,0.085928,0.018949,0.122314
2,0.085554,0.083354,0.090642,0.087402,0.023590,0.122662
3,0.086038,0.082885,0.087621,0.084945,0.013599,0.121269
4,0.083143,0.081015,0.088290,0.086091,0.010959,0.120398
5,0.084429,0.081015,0.086446,0.083963,0.010466,0.120363
6,0.082338,0.079770,0.087283,0.084126,0.010254,0.121338
7,0.082499,0.079145,0.083926,0.080691,0.009251,0.123880
8,0.078642,0.080236,0.083424,0.083144,0.006375,0.126491
9,0.082499,0.083199,0.088123,0.086747,0.021696,0.128684


In [130]:
df1=df
df1.head(5)

,trade_date,open,high,low,close,vol
4436,20000324,3.6733,3.6784,3.5198,3.5198,21221.0
4435,20000327,3.5301,3.6784,3.5301,3.6682,27297.0
4434,20000328,3.7091,3.7501,3.6836,3.7143,33903.0
4433,20000329,3.7245,3.7347,3.5915,3.6375,19684.0
4432,20000330,3.6324,3.6733,3.6119,3.6733,15926.0


In [131]:
t= df1.pop('trade_date')

In [134]:
scaler = MinMaxScaler(feature_range=(0,1))
data = scaler.fit_transform(df1)


array([[0.08442881, 0.08117063, 0.08526827, 0.08118   , 0.01467925],
       [0.07992745, 0.08117063, 0.0856062 , 0.08592763, 0.01894861],
       [0.08555415, 0.08335414, 0.09064244, 0.08740246, 0.02359037],
       ...,
       [0.16385332, 0.15857112, 0.16368922, 0.15860732, 0.04963589],
       [0.15505177, 0.15065323, 0.15909591, 0.15444834, 0.04582666],
       [0.15127968, 0.15613485, 0.16008019, 0.16340614, 0.05611316]])

In [135]:
d=pd.DataFrame(data)

In [138]:
dd=pd.merge(t,d)
dd  

ValueError: can not merge DataFrame with instance of type <class 'pandas.core.series.Series'>